In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 10000
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 10000
N_FEATURES = 12

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 160, 16

In [12]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [13]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 70.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [14]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [15]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [16]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/sub_wise_process_TT/'

In [17]:
for i in range(1,41):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
       
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    print('y_train shape',y_train.shape)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
#     print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 25, 400
    n_depth=12
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
# naveen architecture
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04)), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=8,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(Flatten()))
    # model.add(Flatten())
    model.add(Bidirectional(LSTM(200,return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(200,return_sequences=True)))
    model.add(Dropout(0.3))
    # model.add(Flatten())
    # model.add(Bidirectional(LSTM(200,return_sequences=True)))
    # model.add(Dropout(0.3))
    # model.add(Bidirectional(LSTM(200,return_sequences=True)))
    # model.add(Dropout(0.3))
    # model.add(Flatten())
    # model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    # model.add(Bidirectional(LSTM(200)))
    # model.add(Dropout(0.3))
    model.add(Flatten())
    # model.add(Dense(2000, activation='tanh'))
    model.add(Dense(512, activation='tanh'))
    # model.add(Dense(200, activation='tanh'))
    model.add(BatchNormalization(epsilon=1e-05, momentum=0.9, weights=None))
    model.add(Dense(n_outputs, activation='softmax'))
    #     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
    #     model = Sequential()
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04)), \
    #                            input_shape=(n_steps,n_length,n_depth)))
#     # model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
#     model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
#     # model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Dropout(0.2093)))
#     model.add(TimeDistributed(Activation('relu')))
#     model.add(Flatten())
#     model.add(Dense(512, activation='tanh'))
#     model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
#     model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 1960000

/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 980000

x_train shape:  (195, 10000, 12)
195 training samples
y_train shape:  (195,)
num_time_periods 10000
num_sensors 12
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0]
x_train shape: (10000, 12)
input_shape: (10000, 12)
y_train shape (195,)
New y_train shape:  (195, 49)
x_test shape:  (97, 10000, 12)
97 testing samples
y_test shape:  (97,)
x_train shape:  (195, 25, 400, 12)
x_test shape:  (97, 25, 400, 12)
n_outputs 49
Model: "se

Epoch 10/160
13/13 [==============================] - ETA: 0s - loss: 0.9108 - accuracy: 0.8769
Epoch 00010: val_accuracy improved from 0.76289 to 0.77320, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
13/13 [==============================] - 0s 36ms/step - loss: 0.9108 - accuracy: 0.8769 - val_loss: 1.3895 - val_accuracy: 0.7732 - lr: 0.0010
Epoch 11/160
13/13 [==============================] - ETA: 0s - loss: 0.8848 - accuracy: 0.8667
Epoch 00011: val_accuracy did not improve from 0.77320
13/13 [==============================] - 0s 31ms/step - loss: 0.8848 - accuracy: 0.8667 - val_loss: 1.4301 - val_accuracy: 0.7423 - lr: 0.0010
Epoch 12/160
13/13 [==============================] - ETA: 0s - loss: 0.9050 - accuracy: 0.8462
Epoch 00012: val_accuracy did not improve from 0.77320
13/13 [==============================] - 0s 29ms/step - loss: 0.9050 - accuracy: 0.8462 - val_loss: 1.5621 - val_accuracy: 0.7526 - lr: 0.0010
Epoch 13/160
13/1

Epoch 64/160
11/13 [========================>.....] - ETA: 0s - loss: 0.3581 - accuracy: 1.0000
Epoch 00064: val_accuracy did not improve from 0.90722
13/13 [==============================] - 0s 33ms/step - loss: 0.3626 - accuracy: 1.0000 - val_loss: 0.9719 - val_accuracy: 0.8454 - lr: 0.0010
Epoch 65/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3605 - accuracy: 0.9948
Epoch 00065: val_accuracy did not improve from 0.90722
13/13 [==============================] - 0s 32ms/step - loss: 0.3720 - accuracy: 0.9897 - val_loss: 2.0953 - val_accuracy: 0.5876 - lr: 0.0010
Epoch 66/160
13/13 [==============================] - ETA: 0s - loss: 0.4224 - accuracy: 0.9744
Epoch 00066: val_accuracy did not improve from 0.90722
13/13 [==============================] - 0s 34ms/step - loss: 0.4224 - accuracy: 0.9744 - val_loss: 1.0324 - val_accuracy: 0.8763 - lr: 0.0010
Epoch 67/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3787 - accuracy: 1.0000
Epoch 00067: val_accur

Epoch 92/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3433 - accuracy: 1.0000
Epoch 00092: val_accuracy did not improve from 0.90722
13/13 [==============================] - 0s 34ms/step - loss: 0.3717 - accuracy: 0.9897 - val_loss: 0.9439 - val_accuracy: 0.8969 - lr: 1.0000e-04
Epoch 93/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3439 - accuracy: 1.0000
Epoch 00093: val_accuracy did not improve from 0.90722
13/13 [==============================] - 0s 34ms/step - loss: 0.3461 - accuracy: 1.0000 - val_loss: 0.9336 - val_accuracy: 0.8866 - lr: 1.0000e-04
Epoch 94/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3481 - accuracy: 1.0000
Epoch 00094: val_accuracy did not improve from 0.90722
13/13 [==============================] - 0s 37ms/step - loss: 0.3481 - accuracy: 1.0000 - val_loss: 0.8856 - val_accuracy: 0.8969 - lr: 1.0000e-04
Epoch 95/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3429 - accuracy: 1.0000
Epoch 0009

Epoch 147/160
13/13 [==============================] - ETA: 0s - loss: 0.3298 - accuracy: 1.0000
Epoch 00147: val_accuracy did not improve from 0.91753
13/13 [==============================] - 0s 28ms/step - loss: 0.3298 - accuracy: 1.0000 - val_loss: 0.9704 - val_accuracy: 0.8866 - lr: 1.0000e-05
Epoch 148/160
13/13 [==============================] - ETA: 0s - loss: 0.3307 - accuracy: 1.0000
Epoch 00148: val_accuracy did not improve from 0.91753
13/13 [==============================] - 0s 29ms/step - loss: 0.3307 - accuracy: 1.0000 - val_loss: 0.9634 - val_accuracy: 0.8866 - lr: 1.0000e-05
Epoch 149/160
13/13 [==============================] - ETA: 0s - loss: 0.3281 - accuracy: 1.0000
Epoch 00149: val_accuracy did not improve from 0.91753
13/13 [==============================] - 0s 29ms/step - loss: 0.3281 - accuracy: 1.0000 - val_loss: 0.9621 - val_accuracy: 0.8763 - lr: 1.0000e-05
Epoch 150/160
13/13 [==============================] - ETA: 0s - loss: 0.3327 - accuracy: 1.0000
Epoch 

...

12/13 [==========================>...] - ETA: 0s - loss: 5.3616 - accuracy: 0.0417
Epoch 00001: val_accuracy improved from -inf to 0.02062, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
13/13 [==============================] - 1s 111ms/step - loss: 5.3636 - accuracy: 0.0410 - val_loss: 4.8424 - val_accuracy: 0.0206 - lr: 0.0010
Epoch 2/160
13/13 [==============================] - ETA: 0s - loss: 4.4555 - accuracy: 0.0769
Epoch 00002: val_accuracy improved from 0.02062 to 0.12371, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
13/13 [==============================] - 0s 38ms/step - loss: 4.4555 - accuracy: 0.0769 - val_loss: 4.4204 - val_accuracy: 0.1237 - lr: 0.0010
Epoch 3/160
11/13 [========================>.....] - ETA: 0s - loss: 3.4285 - accuracy: 0.1932
Epoch 00003: val_accuracy improved from 0.12371 to 0.22680, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/C

13/13 [==============================] - 0s 32ms/step - loss: 0.4630 - accuracy: 0.9897 - val_loss: 1.1933 - val_accuracy: 0.7835 - lr: 0.0010
Epoch 26/160
11/13 [========================>.....] - ETA: 0s - loss: 0.4475 - accuracy: 1.0000
Epoch 00026: val_accuracy improved from 0.85567 to 0.86598, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
13/13 [==============================] - 1s 41ms/step - loss: 0.4578 - accuracy: 0.9949 - val_loss: 0.9022 - val_accuracy: 0.8660 - lr: 0.0010
Epoch 27/160
11/13 [========================>.....] - ETA: 0s - loss: 0.4566 - accuracy: 0.9943
Epoch 00027: val_accuracy did not improve from 0.86598
13/13 [==============================] - 0s 31ms/step - loss: 0.4651 - accuracy: 0.9897 - val_loss: 1.0276 - val_accuracy: 0.8351 - lr: 0.0010
Epoch 28/160
13/13 [==============================] - ETA: 0s - loss: 0.4908 - accuracy: 0.9846
Epoch 00028: val_accuracy did not improve from 0.86598
13/13 [==========

Epoch 53/160
11/13 [========================>.....] - ETA: 0s - loss: 0.3800 - accuracy: 1.0000
Epoch 00053: val_accuracy did not improve from 0.87629
13/13 [==============================] - 0s 34ms/step - loss: 0.3800 - accuracy: 1.0000 - val_loss: 1.1853 - val_accuracy: 0.7732 - lr: 0.0010
Epoch 54/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3805 - accuracy: 1.0000
Epoch 00054: val_accuracy did not improve from 0.87629
13/13 [==============================] - 0s 30ms/step - loss: 0.3848 - accuracy: 1.0000 - val_loss: 1.0909 - val_accuracy: 0.7835 - lr: 0.0010
Epoch 55/160
11/13 [========================>.....] - ETA: 0s - loss: 0.4227 - accuracy: 0.9830
Epoch 00055: val_accuracy did not improve from 0.87629
13/13 [==============================] - 0s 33ms/step - loss: 0.4257 - accuracy: 0.9795 - val_loss: 0.9164 - val_accuracy: 0.8144 - lr: 0.0010
Epoch 56/160
11/13 [========================>.....] - ETA: 0s - loss: 0.3905 - accuracy: 0.9943
Epoch 00056: val_accur

Epoch 81/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3563 - accuracy: 1.0000
Epoch 00081: val_accuracy did not improve from 0.88660
13/13 [==============================] - 0s 32ms/step - loss: 0.3564 - accuracy: 1.0000 - val_loss: 0.8674 - val_accuracy: 0.8660 - lr: 1.0000e-04
Epoch 82/160
11/13 [========================>.....] - ETA: 0s - loss: 0.3568 - accuracy: 1.0000
Epoch 00082: val_accuracy did not improve from 0.88660
13/13 [==============================] - 0s 34ms/step - loss: 0.3570 - accuracy: 1.0000 - val_loss: 0.8752 - val_accuracy: 0.8660 - lr: 1.0000e-04
Epoch 83/160
13/13 [==============================] - ETA: 0s - loss: 0.3531 - accuracy: 1.0000
Epoch 00083: val_accuracy did not improve from 0.88660
13/13 [==============================] - 0s 33ms/step - loss: 0.3531 - accuracy: 1.0000 - val_loss: 0.8788 - val_accuracy: 0.8660 - lr: 1.0000e-04
Epoch 84/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3548 - accuracy: 1.0000
Epoch 0008

Epoch 136/160
13/13 [==============================] - ETA: 0s - loss: 0.3357 - accuracy: 1.0000
Epoch 00136: val_accuracy did not improve from 0.89691
13/13 [==============================] - 0s 34ms/step - loss: 0.3357 - accuracy: 1.0000 - val_loss: 0.8486 - val_accuracy: 0.8660 - lr: 1.0000e-04
Epoch 137/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3365 - accuracy: 1.0000
Epoch 00137: val_accuracy did not improve from 0.89691
13/13 [==============================] - 0s 30ms/step - loss: 0.3394 - accuracy: 1.0000 - val_loss: 0.8697 - val_accuracy: 0.8866 - lr: 1.0000e-04
Epoch 138/160
12/13 [==========================>...] - ETA: 0s - loss: 0.3362 - accuracy: 1.0000
Epoch 00138: val_accuracy did not improve from 0.89691
13/13 [==============================] - 0s 31ms/step - loss: 0.3369 - accuracy: 1.0000 - val_loss: 0.8622 - val_accuracy: 0.8866 - lr: 1.0000e-04
Epoch 139/160
13/13 [==============================] - ETA: 0s - loss: 0.3348 - accuracy: 1.0000
Epoch 

In [18]:
statistics.mean(test_acc)

0.8770618483424186